# Data anonymization with Microsoft Presidio

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/extras/guides/privacy/presidio_data_anonymization.ipynb)

## Use case

Data anonymization is crucial before passing information to a language model like GPT-4 because it helps protect privacy and maintain confidentiality. If data is not anonymized, sensitive information such as names, addresses, contact numbers, or other identifiers linked to specific individuals could potentially be learned and misused. Hence, by obscuring or removing this personally identifiable information (PII), data can be used freely without compromising individuals' privacy rights or breaching data protection laws and regulations.

## Overview

Anonynization consists of two steps:

1. **Identification:** Identify all data fields that contain personally identifiable information (PII).
2. **Replacement**: Replace all PIIs with pseudo values or codes that do not reveal any personal information about the individual but can be used for reference. We're not using regular encryption, because the language model won't be able to understand the meaning or context of the encrypted data.

We use *Microsoft Presidio* together with *Faker* framework for anonymization purposes because of the wide range of functionalities they provide. The full implementation is available in `PresidioAnonymizer`.

## Quickstart

Below you will find the use case on how to leverage anonymization in LangChain.

In [1]:
# Install necessary packages
# ! pip install langchain langchain-experimental openai
# ! python -m spacy download en_core_web_lg

\
Let's see how PII anonymization works using a sample sentence:

In [2]:
from langchain_experimental.data_anonymizer import PresidioAnonymizer

anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON"])

anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

'My name is Marie Santos, call me at 313-666-7440 or email me at real.slim.shady@gmail.com'

\
As can be observed, the name was correctly identified and replaced with another. The `analyzed_fields` attribute is responsible for what values are to be detected and substituted. We can add *PHONE_NUMBER* to the list:

In [3]:
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON", "PHONE_NUMBER"])
anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

'My name is Victoria Mckinney, call me at 713-549-8623 or email me at real.slim.shady@gmail.com'

\
If no analyzed_fields are specified, by default the anonymizer will detect all supported formats. Below is the full list of them:

`['PERSON', 'EMAIL_ADDRESS', 'PHONE_NUMBER', 'IBAN_CODE', 'CREDIT_CARD', 'CRYPTO', 'IP_ADDRESS', 'LOCATION', 'DATE_TIME', 'NRP', 'MEDICAL_LICENSE', 'URL', 'US_BANK_NUMBER', 'US_DRIVER_LICENSE', 'US_ITIN', 'US_PASSPORT', 'US_SSN']`

**Disclaimer:** We suggest carefully defining the private data to be detected - Presidio doesn't work perfectly and it sometimes makes mistakes, so it's better to have more control over the data.

In [4]:
anonymizer = PresidioAnonymizer()
anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

'My name is Billy Russo, call me at 970-996-9453x038 or email me at jamie80@example.org'

\
It may be that the above list of detected fields is not sufficient. For example, the already available *PHONE_NUMBER* field does not support polish phone numbers and confuses it with another field:

In [5]:
anonymizer = PresidioAnonymizer()
anonymizer.anonymize("My polish phone number is 666555444")

'My polish phone number is EVIA70648911396944'

\
You can then write your own recognizers and add them to the pool of those present. How exactly to create recognizers is described in the [Presidio documentation](https://microsoft.github.io/presidio/samples/python/customizing_presidio_analyzer/).

In [6]:
# Define the regex pattern in a Presidio `Pattern` object:
from presidio_analyzer import Pattern, PatternRecognizer


polish_phone_numbers_pattern = Pattern(
    name="polish_phone_numbers_pattern",
    regex="(?<!\w)(\(?(\+|00)?48\)?)?[ -]?\d{3}[ -]?\d{3}[ -]?\d{3}(?!\w)",
    score=1,
)

# Define the recognizer with one or more patterns
polish_phone_numbers_recognizer = PatternRecognizer(
    supported_entity="POLISH_PHONE_NUMBER", patterns=[polish_phone_numbers_pattern]
)

\
Now, we can add recognizer by calling `add_recognizer` method on the anonymizer:

In [7]:
anonymizer.add_recognizer(polish_phone_numbers_recognizer)

\
And voilà! With the added pattern-based recognizer, the anonymizer now handles polish phone numbers.

In [8]:
print(anonymizer.anonymize("My polish phone number is 666555444"))
print(anonymizer.anonymize("My polish phone number is 666 555 444"))
print(anonymizer.anonymize("My polish phone number is +48 666 555 444"))

My polish phone number is <POLISH_PHONE_NUMBER>
My polish phone number is <POLISH_PHONE_NUMBER>
My polish phone number is <POLISH_PHONE_NUMBER>


\
The problem is - even though we recognize polish phone numbers now, we don't have a method (operator) that would tell how to substitute a given field - because of this, in the outpit we only provide string `<POLISH_PHONE_NUMBER>` We need to create a method to replace it correctly: 

In [9]:
from faker import Faker

fake = Faker(locale="pl_PL")


def fake_polish_phone_number(_=None):
    return fake.phone_number()


fake_polish_phone_number()

'+48 533 220 543'

\
We used Faker to create pseudo data. Now we can create an operator and add it to the anonymizer. For complete information about operators and their creation, see the Presidio documentation for [simple](https://microsoft.github.io/presidio/tutorial/10_simple_anonymization/) and [custom](https://microsoft.github.io/presidio/tutorial/11_custom_anonymization/) anonymization.

In [10]:
from presidio_anonymizer.entities import OperatorConfig

new_operators = {
    "POLISH_PHONE_NUMBER": OperatorConfig(
        "custom", {"lambda": fake_polish_phone_number}
    )
}

In [11]:
anonymizer.add_operators(new_operators)

In [12]:
anonymizer.anonymize("My polish phone number is 666555444")

'My polish phone number is +48 692 715 636'

\
Finally, it is worth showing how to implement anonymizer as a chain. Since anonymization is based on string operations, we can use `TransformChain` for this:

In [13]:
from langchain.chains.transform import TransformChain

anonymizer = PresidioAnonymizer()


def anonymize_func(inputs: dict) -> dict:
    text = inputs["text"]
    return {"anonymized_text": anonymizer.anonymize(text)}


anonymize_chain = TransformChain(
    input_variables=["text"],
    output_variables=["anonymized_text"],
    transform=anonymize_func,
)

anonymize_chain("You can find our super secret data at https://supersecretdata.com")

{'text': 'You can find our super secret data at https://supersecretdata.com',
 'anonymized_text': 'You can find our super secret data at https://www.fox.org/'}

\
Later, you can, for example, use such anonymization as part of chain sequence. We will use `LangChain Expression Language` ([learn more here](https://python.langchain.com/docs/guides/expression_language/)) for composing these chains together, as shown below:

In [14]:
# ! pip install openai

# Set env var OPENAI_API_KEY or load from a .env file:
import dotenv

dotenv.load_dotenv()

True

In [17]:
from operator import itemgetter
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.llms.openai import OpenAI

template = """According to this text, where can you find our super secret data?

{anonymized_text}

Answer:"""
prompt = PromptTemplate(input_variables=["anonymized_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)


chain = (
    anonymize_chain
    | {"anonymized_text": itemgetter("anonymized_text")}
    | prompt
    | llm_chain
)
chain.invoke("You can find our super secret data at https://supersecretdata.com")

{'anonymized_text': StringPromptValue(text='According to this text, where can you find our super secret data?\n\nYou can find our super secret data at https://evans-summers.info/\n\nAnswer:'),
 'text': ' https://evans-summers.info/'}

## Future works

- **deanonymization** - add the ability to reverse anonymization. For example, the workflow could look like this: `anonymize -> LLMChain -> deanonymize`. By doing this, we will retain anonymity in requests to, for example, OpenAI, and then be able restore the original data.
- **instance anonymization** - at this point, each occurrence of PII is treated as a separate entity and separately anonymized. Therefore, two occurrences of the name John Doe in the text will be changed to two different names. It is therefore worth introducing support for full instance detection, so that repeated occurrences are treated as a single object.